# Anayzing Public Data



## INIT

In [2]:
import pandas as pd
from util.build_dataset import build_origin_price

In [3]:
# metropolitan = ['서울서부', '대전', '부산', '대구', '광주', '인천', '울산', '제주']

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 200)

In [5]:
origin_df, key = build_origin_price(bucket_name="production-bobsim", date="201908")

2020-04-09 02:34:48:manage:<INFO> 1 files is loaded from public_data/open_data_raw_material_price/origin/csv/201908.csv in s3 'production-bobsim'


In [6]:
origin_df.columns
origin_df.head(3)

,date,region,unit_name,grade,class,standard_item_name,price
0,2019-08-01,서울서부,20KG,상(1등급),소비자가격,벼,54300
1,2019-08-01,서울서부,20KG,상(1등급),소비자가격,벼,53200
2,2019-08-01,서울서부,20KG,상(1등급),소비자가격,벼,50600


In [7]:
df = origin_df["standard_item_name"].drop_duplicates()
df.count()

76

In [8]:
group = origin_df.groupby(["standard_item_name"]).count()["date"].to_frame()/origin_df["date"].count()*100

In [9]:
group[group.apply(lambda x: x<1)].dropna().info()

<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, 갈치류 to 호두
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    33 non-null     float64
dtypes: float64(1)
memory usage: 528.0+ bytes


## Region

In [10]:
origin_df.groupby("region").count().info()

<class 'pandas.core.frame.DataFrame'>
Index: 95 entries, 강원 to 충북충주
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   date                95 non-null     int64
 1   unit_name           95 non-null     int64
 2   grade               95 non-null     int64
 3   class               95 non-null     int64
 4   standard_item_name  95 non-null     int64
 5   price               95 non-null     int64
dtypes: int64(6)
memory usage: 5.2+ KB


In [ ]:
 origin_df[origin_df["class"] == "소비자가격"]

In [17]:
region = origin_df[origin_df["class"] == "소비자가격"].groupby(["region"]).count()["date"].to_frame()/origin_df["date"].count()*100
print(region.index)
region

Index(['강원', '강원강릉', '강원춘천', '경기', '경기수원', '경기의정부', '경남창원', '경북안동', '경북포항',
       '광주', '대구', '대전', '부산', '서울', '서울서부', '세종', '울산', '인천', '전남순천', '전북전주',
       '전주', '제주', '청주', '충북청주'],
      dtype='object', name='region')


,date
region,
강원,1.336564
강원강릉,3.500524
강원춘천,1.826226
경기,1.500811
경기수원,2.452420
경기의정부,1.628103
경남창원,3.975815
경북안동,1.604492
경북포항,3.173055


In [16]:
region.index

Index(['강원', '강원강릉', '강원춘천', '경기', '경기수원', '경기의정부', '경남창원', '경북안동', '경북포항',
       '광주', '대구', '대전', '부산', '서울', '서울서부', '세종', '울산', '인천', '전남순천', '전북전주',
       '전주', '제주', '청주', '충북청주'],
      dtype='object', name='region')

In [122]:
region[region.apply(lambda x: x<5)].dropna().info()

<class 'pandas.core.frame.DataFrame'>
Index: 89 entries, 강원 to 충북충주
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    89 non-null     float64
dtypes: float64(1)
memory usage: 1.4+ KB


In [123]:
geography = {
    (
        '강원', '강원강릉', '강원삼척', '강원양양',
        '강원원주', '강원정선', '강원춘천', '강원평창',
        '강원홍천', '강원횡성'
    ): "강원도",
    (
        '서울', '서울서부', '인천', '제주',
        '경기', '경기수원', '경기안성', '경기양주',
        '경기양평', '경기의정부', '경기이천', '경기파주',
        '경기평택', '경기포천', '경기화성'
    ): "서울경기",
    (
        '경남거창', '경남고성', '경남남해', '경남사천', '경남양산', '경남울산',
        '경남진주', '경남창녕', '경남창원', '경남하동', '경남함안', '경남함양', '경남합천', '경북경산', '경북경주',
        '경북고령', '경북구미', '경북김천', '경북봉화', '경북상주', '경북안동', '경북영양', '경북영주', '경북영천',
        '경북예천', '경북의성', '경북청도', '경북청송', '경북포항',
        '대구', '부산', '울산'
    ): "경상도",
    (
        '전남강진', '전남고흥', '전남나주', '전남담양', '전남무안',
        '전남순천', '전남영광', '전남장성', '전남함평', '전남화순', '전북김제', '전북남원', '전북부안', '전북순창',
        '전북임실', '전북장수', '전북전주', '전북정읍', '전주', '광주'
    ): "전라도",
    (
        '충남공주', '충남논산',
        '충남부여', '충남서산', '충남아산', '충남예산', '충남천안', '충남홍성', '충북보은', '충북영동', '충북옥천',
        '충북음성', '충북제천', '충북청주', '충북충주',
        '청주', '대전', '세종'
    ): "충청도"
}

geo = {
    "강원도": [
        '강원', '강원강릉', '강원삼척', '강원양양',
        '강원원주', '강원정선', '강원춘천', '강원평창',
        '강원홍천', '강원횡성'
    ],
    "경기도": [
        '서울', '서울서부', '인천', '제주',
        '경기', '경기수원', '경기안성', '경기양주',
        '경기양평', '경기의정부', '경기이천', '경기파주',
        '경기평택', '경기포천', '경기화성'
    ],
    "경상도": [
        '경남거창', '경남고성', '경남남해', '경남사천', '경남양산', '경남울산',
        '경남진주', '경남창녕', '경남창원', '경남하동', '경남함안', '경남함양', '경남합천', '경북경산', '경북경주',
        '경북고령', '경북구미', '경북김천', '경북봉화', '경북상주', '경북안동', '경북영양', '경북영주', '경북영천',
        '경북예천', '경북의성', '경북청도', '경북청송', '경북포항',
        '대구', '부산', '울산'
    ],
    "전라도": [
        '전남강진', '전남고흥', '전남나주', '전남담양', '전남무안',
        '전남순천', '전남영광', '전남장성', '전남함평', '전남화순', '전북김제', '전북남원', '전북부안', '전북순창',
        '전북임실', '전북장수', '전북전주', '전북정읍', '전주', '광주'
    ],
    "충청도": [
        '충남공주', '충남논산',
        '충남부여', '충남서산', '충남아산', '충남예산', '충남천안', '충남홍성', '충북보은', '충북영동', '충북옥천',
        '충북음성', '충북제천', '충북청주', '충북충주',
        '청주', '대전', '세종'
    ]
}


In [124]:
def find_geo(region):
    for key in geography.keys():
        if region in key:
            return geography[key]

In [129]:
origin_df.region.apply(find_geo)

0        서울경기
1        서울경기
2        서울경기
3        서울경기
4        서울경기
5        서울경기
6        서울경기
7        서울경기
8        서울경기
9        서울경기
10        경상도
11        경상도
12        경상도
13        경상도
14        경상도
15        경상도
16        경상도
17        경상도
18        경상도
19        경상도
20       서울경기
21       서울경기
22        전라도
23        전라도
24        전라도
25        전라도
26        전라도
27        충청도
28        충청도
29        충청도
30        충청도
31        충청도
32        경상도
33        경상도
34        충청도
35       서울경기
36       서울경기
37       서울경기
38        강원도
39        강원도
40        강원도
41        강원도
42        충청도
43        충청도
44        전라도
45        전라도
46        전라도
47        전라도
48        경상도
49        경상도
50        경상도
51        경상도
52        경상도
53       서울경기
54       서울경기
55       서울경기
56       서울경기
57       서울경기
58       서울경기
59       서울경기
60       서울경기
61       서울경기
62       서울경기
63       서울경기
64       서울경기
65        경상도
66        경상도
67        경상도
68        경상도
69        경상도
70        경상도
71    

In [125]:
region = origin_df.apply(lambda x: find_geo(x.region), axis=1)

In [126]:
count = region.to_frame().rename(columns={0:"region"}).reset_index().groupby("region").count()
count

,index
region,
강원도,6583
경상도,30111
서울경기,30362
전라도,16559
충청도,13799


In [127]:
count/count.sum()*100

,index
region,
강원도,6.757756
경상도,30.910341
서울경기,31.168005
전라도,16.998583
충청도,14.165315


In [128]:
fuck = origin_df.groupby(['표준품목명']).count()["조사일자"].to_frame()
print(fuck.info())
fuck/fuck.sum()

KeyError: '표준품목명'

## Was the item evenly researched according to region or date?


In [ ]:
# First, let's think about the consumer price, not the wholesale price
df = origin_df
df = df[(df.조사구분명 == '소비자가격')] # & (df.조사일자 == '2019-01-04')]
print(df.head(2))
df = df[['조사일자', '표준품목명', '조사가격품목명', '표준품종명', '조사가격품종명', '조사지역명', '표준시장명']]

In [16]:
c_df = df.groupby(["조사일자", "조사지역명", "표준품목명", "조사가격품목명", "조사가격품종명", "표준품종명"]).count()
c_df.rename(columns = {'표준시장명': 'm_count'}, inplace=True)
c_df.head(100)
# l_df.index

# m_count: count the number of markets by each item.

m_count
조사일자       조사지역명 표준품목명    조사가격품목명 조사가격품종명   표준품종명             
2019-08-01 강원    감자       감자      수미        수미               1
                 건가공(포)류  건오징어    일반        오징어포             1
                 건가공(해조)류 건미역     일반        자연미역             1
                 건제품      건포도     수입        건포도(수입)          1
                 고구마      고구마(밤)  고구마(일반)   밤고구마             1
                 고추       붉은고추    일반        홍고추              1
                          풋고추     꽈리고추      꽈리고추             1
                                  청양고추      청양               1
                 김류       김       일반        김                1
                 깻잎       깻잎      일반        깻잎(일반)           1
                 당근       당근      무세척       흙당근              1
                 대파       파       대파        대파(일반)           1
                 돈육       돼지고기    목살        목심살              1
                                  삼겹살(국산냉장) 삼겹살              1
                                  앞다리살      앞다리살             1
                 땅콩       땅콩      국산        땅콩(일반)           1
                                  수입        땅콩(수입)           1
                 레몬       레몬      수입        레몬(수입)           1
                 메론       멜론      일반        네트계              1
                 무        무       고랭지       고냉지무             1
                 바나나      바나나     수입        바나나(수입)          1
                 배        배       신고        신고               1
                 배추       배추      고랭지       고냉지배추            1
                          얼갈이배추   일반        얼갈이배추            1
                 벼        쌀       일반계       일반계              1
                 블루베리     건블루베리   수입        건블루베리(수입)        1
                 사과       사과      후지        후지               1
                 상추       상추      적         적상추              1
                 새송이      새송이버섯   일반        새송이버섯            1
                 생강       생강      국산        생강(일반)           1
                 쇠고기      쇠고기     호주산갈비     갈비(수입)           1
                 수박       수박      일반        수박(일반)           1
                 아몬드      아몬드     일반        아몬드(수입)          1
                 양배추      양배추     일반        양배추(일반)          1
                 양파       양파      양파(일반)    기타               1
                 열무       열무      일반        열무(일반)           1
                 오이       오이      다다기계통     백다다기             1
                                  취청        취청               1
                 우유       우유      일반        우유(일반)           1
                 전복류      전복      일반        전복               1
                 젓갈(액젓)류  멸치액젓    일반        멸치액젓             1
                 조기류      조기      수입부세      참조기              1
                 조란       계란      일반        특란               1
                 조미제품     고춧가루    일반        고추가루             1
                 참다래(키위)  참다래     수입        키위(수입)           1
                 참외       참외      일반        금싸라기             1
                 찹쌀       찹쌀      일반찹쌀      일반계              1
                 천연조미료류   굵은소금    일반        천일염              1
                 체리       체리      수입        체리(수입)           1
                 콩        콩       백태(국산)    백태               1
                 파프리카     파프리카    일반        기타               1
                 팥        팥(적두)   국산        붉은팥              1
                 팽이버섯     팽이버섯    일반        팽이1호             1
                 포도       포도      거봉        거봉               1
                                  캠벨얼리      캠벨얼리             1
                 피망(단고추)  피망      청피망       청피망              1
                 한우       쇠고기     양지        양지               1
                                  한우등심      등심               2
                 호두       호두      일반        호두(일반)           1
                 호박       호박      애호박       애호박              1
                                  쥬키니       쥬키니호박            1
           강원강릉  갈치류      갈치      냉동        은갈치              1
                       

In [362]:
l_df = c_df.groupby(["조사일자", "조사지역명"]).count()
l_df.rename(columns = {'m_count': 'i_count'}, inplace=True)
# s_df = tmp_df.groupby(["조사지역명"]).sum()
l_df.head(48)

# p_count: count the number of items by each region

i_count
조사일자       조사지역명         
2017-01-02 강원          67
           강원강릉        98
           강원춘천        70
           경기          71
           경기수원        93
           경기의정부       82
           경남창원        95
           경북안동        76
           경북포항        97
           광주         102
           대구         100
           대전         104
           부산         105
           서울          95
           서울서부       103
           세종          67
           울산          92
           인천          98
           전남순천       103
           전북전주        92
           전주          71
           제주          93
           청주          62
           충북청주        88
2017-01-03 강원          69
           강원강릉        98
           강원춘천        70
           경기          71
           경기수원        92
           경기의정부       83
           경남창원        95
           경북안동        78
           경북포항        98
           광주         102
           대구         100
           대전         104
           부산         105
           서울          96
           서울서부       103
           세종          70
           울산          92
           인천          98
           전남순천       103
           전북전주        92
           전주          71
           제주          95
           청주          62
           충북청주        88

In [363]:
date_df = l_df.groupby("조사일자").count()
date_df.rename(columns = {'i_count': 'r_count'}, inplace=True)

r_df = l_df.drop(['i_count'], axis=1).reset_index(inplace = False)
date_df["regions"] = r_df.groupby("조사일자")["조사지역명"].apply(list)

# date_df["count"] = l_df.groupby("조사일자").count()
date_df["p_sum"] = l_df.groupby("조사일자").sum()
date_df

# r_count: count the number of regions by date
# regions: list of region that investigated on each date.
# p_sum: count the sum of items by region on each date.

,r_count,regions,p_sum
조사일자,,,
2017-01-02,24,"[강원, 강원강릉, 강원춘천, 경기, 경기수원, 경기의정부, 경남창원, 경북안동, 경북포항, 광주, 대구, 대전, 부산, 서울, 서울서부, 세종, 울산, 인천, 전남순천, 전북전주, 전주, 제주, 청주, 충북청주]",2124
2017-01-03,24,"[강원, 강원강릉, 강원춘천, 경기, 경기수원, 경기의정부, 경남창원, 경북안동, 경북포항, 광주, 대구, 대전, 부산, 서울, 서울서부, 세종, 울산, 인천, 전남순천, 전북전주, 전주, 제주, 청주, 충북청주]",2135
2017-01-04,24,"[강원, 강원강릉, 강원춘천, 경기, 경기수원, 경기의정부, 경남창원, 경북안동, 경북포항, 광주, 대구, 대전, 부산, 서울, 서울서부, 세종, 울산, 인천, 전남순천, 전북전주, 전주, 제주, 청주, 충북청주]",2150
2017-01-05,24,"[강원, 강원강릉, 강원춘천, 경기, 경기수원, 경기의정부, 경남창원, 경북안동, 경북포항, 광주, 대구, 대전, 부산, 서울, 서울서부, 세종, 울산, 인천, 전남순천, 전북전주, 전주, 제주, 청주, 충북청주]",2138
2017-01-06,24,"[강원, 강원강릉, 강원춘천, 경기, 경기수원, 경기의정부, 경남창원, 경북안동, 경북포항, 광주, 대구, 대전, 부산, 서울, 서울서부, 세종, 울산, 인천, 전남순천, 전북전주, 전주, 제주, 청주, 충북청주]",2151
2017-01-09,24,"[강원, 강원강릉, 강원춘천, 경기, 경기수원, 경기의정부, 경남창원, 경북안동, 경북포항, 광주, 대구, 대전, 부산, 서울, 서울서부, 세종, 울산, 인천, 전남순천, 전북전주, 전주, 제주, 청주, 충북청주]",2149
2017-01-10,24,"[강원, 강원강릉, 강원춘천, 경기, 경기수원, 경기의정부, 경남창원, 경북안동, 경북포항, 광주, 대구, 대전, 부산, 서울, 서울서부, 세종, 울산, 인천, 전남순천, 전북전주, 전주, 제주, 청주, 충북청주]",2147
2017-01-12,24,"[강원, 강원강릉, 강원춘천, 경기, 경기수원, 경기의정부, 경남창원, 경북안동, 경북포항, 광주, 대구, 대전, 부산, 서울, 서울서부, 세종, 울산, 인천, 전남순천, 전북전주, 전주, 제주, 청주, 충북청주]",2146
2017-01-13,24,"[강원, 강원강릉, 강원춘천, 경기, 경기수원, 경기의정부, 경남창원, 경북안동, 경북포항, 광주, 대구, 대전, 부산, 서울, 서울서부, 세종, 울산, 인천, 전남순천, 전북전주, 전주, 제주, 청주, 충북청주]",2152


**강원, 강원강릉, 강원춘천, 경기, 경기수원, 경기의정부, 경남창원, 경북안동, 경북포항, 광주, 대구, 대전, 부산, 서울, 서울서부, 세종, 울산, 인천, 전남순천, 전북전주, 전주, 제주, 청주, 충북청주**


* 수도권(6)
    * 서울(2)
        * 서울, 서울서부

    * 인천(1)
        * 인천

    * 경기(3)
        * 경기, 경기수원, 경기의정부
    
* 강원도(3)
    * 강원, 강원강릉, 강원춘천
    
* 충청도(4)
    * 대전, 청주, 충북청주, 세종
    
* 전라도(4)
    * 광주, 전남순천, 전북전주, 전주
    
* 경상도(6)
    * 경남창원, 경북안동, 경북포항, 대구, 부산, 울산
    
* 제주도(1)
    * 제주

## Price distribution map by region

### Which 'by distance avg' or 'by nearest dist' should i calculate item price.


#### region

#### market

#### distributor

In [440]:
df = origin_df
df = df[(df.조사구분명 == '소비자가격')] #  & (df.조사일자 == '2017-01-04')]
df = df[['조사일자', # date
         '표준품목명', '조사가격품목명', '표준품종명', '조사가격품종명', # item category
         '조사지역명', '표준시장명', '조사가격시장명', # region, market, distributor
         '당일조사가격', '전일조사가격' # price (today and previous day)
        ]]

In [452]:
# constant
target_group = '조사지역명'
target_item = "쇠고기"

In [453]:
# region_df = df.groupby(['조사지역명', '표준품목명', '조사가격품목명', '표준품종명', '조사가격품종명']).mean()
# region_df.reset_index(inplace=True)
# region_df[region_df.표준품목명 == target_item]

t_df = df[df.표준품목명 == target_item]
g_df = t_df.groupby('조사지역명')
stat = g_df.describe()
# stat.keys()
result = stat.sort_values(by=[('당일조사가격', 'std'),('당일조사가격','mean')], ascending=False, axis=0)
result.describe()
# r_df = df[region_df.표준품목명 == target_item]

당일조사가격                                                      \
            count         mean          std          min          25%   
count   22.000000    22.000000    22.000000    22.000000    22.000000   
mean    95.818182  2201.346354   547.839871  1562.272727  1801.022727   
std     66.767997   353.883883   395.470374   291.317726   319.507302   
min     11.000000  1486.666667    15.201351  1300.000000  1330.000000   
25%     41.750000  1975.676692   293.055761  1390.000000  1560.625000   
50%     72.000000  2221.419703   475.492775  1470.000000  1690.000000   
75%    119.750000  2418.924107   733.056593  1607.500000  2035.000000   
max    266.000000  2833.218391  1378.770145  2390.000000  2590.000000   

                                                 전일조사가격               \
               50%         75%          max       count         mean   
count    22.000000    22.00000    22.000000   22.000000    22.000000   
mean   2083.636364  2508.75000  3430.454545   95.818182  2202.525192   
std     294.008349   626.27097  1250.769201   66.767997   353.121793   
min    1430.000000  1700.00000  1700.000000   11.000000  1486.666667   
25%    1890.000000  2220.00000  2550.000000   41.750000  1971.951570   
50%    2080.000000  2500.00000  2990.000000   72.000000  2211.541161   
75%    2287.500000  2590.00000  4972.500000  119.750000  2429.593750   
max    2590.000000  4950.00000  5480.000000  266.000000  2822.873563   

                                                                        \
               std          min          25%          50%          75%   
count    22.000000    22.000000    22.000000    22.000000    22.000000   
mean    547.489802  1571.363636  1801.022727  2088.636364  2515.340909   
std     395.250882   320.078792   319.507302   320.669619   625.378122   
min      15.201351  1300.000000  1330.000000  1430.000000  1700.000000   
25%     302.274275  1390.000000  1560.625000  1882.500000  2220.000000   
50%     466.507778  1470.000000  1690.000000  2080.000000  2522.500000   
75%     719.978580  1607.500000  2035.000000  2287.500000  2590.000000   
max    1378.770145  2590.000000  2590.000000  2700.000000  4950.000000   

                    
               max  
count    22.000000  
mean   3433.636364  
std    1246.146615  
min    1700.000000  
25%    2575.000000  
50%    2990.000000  
75%    4972.500000  
max    5480.000000

In [460]:
t_df = df[df.표준품목명 == target_item]
g_df = t_df.groupby(['조사지역명', '표준시장명'])
stat = g_df.describe()
# stat.keys()
result = stat.sort_values(by=[('당일조사가격', 'std'),('당일조사가격','mean')], ascending=False, axis=0)
result # .describe()

당일조사가격                                            \
                          count         mean          std     min     25%   
조사지역명 표준시장명                                                                 
경남창원  이마트 창원점              15.0  4250.000000  1571.009684  2200.0  2475.0   
제주    이마트 신제주점             50.0  3133.800000  1398.822960  1980.0  2080.0   
대전    이마트 둔산점              58.0  3067.931034  1353.575847  1680.0  2080.0   
서울    홈플러스 금천점             48.0  2844.166667  1345.593017  1490.0  1590.0   
부산    메가마트 동래점             62.0  2646.129032  1178.070602  1380.0  1980.0   
경북안동  이마트 안동점              87.0  2833.218391  1155.234290  1680.0  2200.0   
서울    이마트 수색점              54.0  2297.037037   759.427862  1680.0  1980.0   
대전    롯데마트 노은점             50.0  2199.400000   685.706577  1390.0  1660.0   
대구    이마트 반야월점             60.0  2476.500000   653.563440  1900.0  2080.0   
광주    양동시장                 76.0  1893.421053   552.167976  1300.0  1375.0   
강원강릉  홈플러스 강릉점             50.0  2510.000000   549.210937  1490.0  2590.0   
경북포항  롯데마트 포항점             54.0  2007.037037   545.664316  1390.0  1610.0   
광주    홈플러스 하남점             57.0  2491.754386   542.971726  1490.0  2490.0   
대구    홈플러스 내당점             52.0  2361.153846   527.047469  1490.0  1890.0   
부산    홈플러스 가야점             38.0  2200.526316   507.665283  1490.0  1590.0   
경기의정부 홈플러스 의정부             57.0  2088.245614   483.473750  1390.0  1590.0   
충북청주  육거리종합시장              38.0  1969.473684   467.511800  1500.0  1500.0   
인천    롯데마트 부평점             44.0  2067.727273   451.595948  1390.0  1640.0   
서울    롯데마트 잠실점             50.0  2255.400000   434.629610  1610.0  1960.0   
강원    롯데마트 춘천석사            35.0  2406.857143   419.325589  1610.0  1960.0   
전남순천  역전시장                 76.0  1866.447368   402.756184  1450.0  1630.0   
      홈플러스 순천점             43.0  2220.232558   384.543566  1490.0  1890.0   
강원강릉  강릉중앙시장               57.0  1833.333333   371.451464  1400.0  1400.0   
서울    롯데마트 청량리점            32.0  2075.937500   366.763169  1640.0  1880.0   
서울서부  경동시장                 95.0  1806.000000   355.463138  1400.0  1450.0   
경남창원  상남시장(상남시장(상설+5일장))   38.0  1880.000000   324.295494  1560.0  1560.0   
대전    역전시장                 76.0  1813.157895   319.725760  1500.0  1600.0   
서울서부  영등포 유통상가            114.0  1766.666667   299.458705  1300.0  1500.0   
전주    이마트                  38.0  2529.210526   286.303388  1980.0  2280.0   
인천    현대시장                 76.0  1787.368421   282.896299  1500.0  1545.0   
부산    부전시장                 95.0  1980.000000   263.305502  1800.0  1850.0   
서울서부  복조리시장                57.0  1876.666667   251.526293  1580.0  1580.0   
제주    동문시장                 18.0  1930.000000   241.904211  1600.0  1600.0   
광주    이마트 상무점              38.0  2411.842105   201.693576  1980.0  2280.0   
경기수원  지동시장                 76.0  1688.684211   177.556317  1350.0  1562.5   
전북전주  전주남부시장               57.0  1486.666667   157.665257  1330.0  1330.0   
울산    홈플러스 남구점             38.0  2595.263158    73.327729  2390.0  2590.0   
세종    이마트 세종점              11.0  2236.363636    41.778637  2200.0  2200.0   
강원춘천  춘천중앙시장               38.0  1815.000000    15.201351  1800.0  1800.0   

                                                 전일조사가격               \
                             50%     75%     max  count         mean   
조사지역명 표준시장명                                                            
경남창원  이마트 창원점             5480.0  5480.0  5480.0   15.0  4250.000000   
제주    이마트 신제주점            2080.0  4972.5  4980.0   50.0  3133.200000   
대전    이마트 둔산점             2280.0  4980.0  4980.0   58.0  3059.310345   
서울    홈플러스 금천점            2590.0  2590.0  5290.0   48.0  2848.333333   
부산    메가마트 동래점            1980.0  2980.0  4980.0   62.0  2613.870968   
경북안동  이마트 안동점             2280.0  2830.0  4980.0   87.0  2822.873563   
서울    이마트 수색점             1980.0  2580.0  5280.0   54.0  2285.925926   
대전    롯데마트 노은점            1980.0  2650.0  3950.0   50.0  2237.400000  

In [459]:
t_df = df[df.표준품목명 == target_item]
g_df = t_df.groupby(['조사지역명','조사가격시장명'])
stat = g_df.describe()
# stat.keys()
result = stat.sort_values(by=[('당일조사가격', 'std'),('당일조사가격','mean')], ascending=False, axis=0)
result
# result.describe()

당일조사가격                                                    \
               count         mean          std     min     25%     50%   
조사지역명 조사가격시장명                                                            
경남창원  C-유통      15.0  4250.000000  1571.009684  2200.0  2475.0  5480.0   
제주    C-유통      50.0  3133.800000  1398.822960  1980.0  2080.0  2080.0   
대전    C-유통      58.0  3067.931034  1353.575847  1680.0  2080.0  2280.0   
서울    A-유통      48.0  2844.166667  1345.593017  1490.0  1590.0  2590.0   
부산    D-유통      62.0  2646.129032  1178.070602  1380.0  1980.0  1980.0   
경북안동  C-유통      87.0  2833.218391  1155.234290  1680.0  2200.0  2280.0   
서울    C-유통      54.0  2297.037037   759.427862  1680.0  1980.0  1980.0   
대전    E-유통      50.0  2199.400000   685.706577  1390.0  1660.0  1980.0   
대구    C-유통      60.0  2476.500000   653.563440  1900.0  2080.0  2280.0   
광주    양동시장      76.0  1893.421053   552.167976  1300.0  1375.0  1850.0   
강원강릉  A-유통      50.0  2510.000000   549.210937  1490.0  2590.0  2590.0   
경북포항  E-유통      54.0  2007.037037   545.664316  1390.0  1610.0  1880.0   
광주    A-유통      57.0  2491.754386   542.971726  1490.0  2490.0  2690.0   
대구    A-유통      52.0  2361.153846   527.047469  1490.0  1890.0  2590.0   
부산    A-유통      38.0  2200.526316   507.665283  1490.0  1590.0  2490.0   
경기의정부 A-유통      57.0  2088.245614   483.473750  1390.0  1590.0  1890.0   
충북청주  육거리시장     38.0  1969.473684   467.511800  1500.0  1500.0  1980.0   
인천    E-유통      44.0  2067.727273   451.595948  1390.0  1640.0  2030.0   
서울    E-유통      50.0  2255.400000   434.629610  1610.0  1960.0  2100.0   
강원    E-유통      35.0  2406.857143   419.325589  1610.0  1960.0  2500.0   
전남순천  역전시장      76.0  1866.447368   402.756184  1450.0  1630.0  1680.0   
      A-유통      43.0  2220.232558   384.543566  1490.0  1890.0  1990.0   
강원강릉  중앙시장      57.0  1833.333333   371.451464  1400.0  1400.0  1800.0   
서울    F-유통      32.0  2075.937500   366.763169  1640.0  1880.0  1980.0   
서울서부  경동시장      95.0  1806.000000   355.463138  1400.0  1450.0  1760.0   
경남창원  창원상남시장    38.0  1880.000000   324.295494  1560.0  1560.0  1880.0   
대전    역전시장      76.0  1813.157895   319.725760  1500.0  1600.0  1700.0   
서울서부  영등포시장    114.0  1766.666667   299.458705  1300.0  1500.0  1800.0   
전주    C-유통      38.0  2529.210526   286.303388  1980.0  2280.0  2480.0   
인천    현대시장      76.0  1787.368421   282.896299  1500.0  1545.0  1680.0   
부산    부전시장      95.0  1980.000000   263.305502  1800.0  1850.0  1850.0   
서울서부  복조리시장     57.0  1876.666667   251.526293  1580.0  1580.0  1860.0   
제주    동문시장      18.0  1930.000000   241.904211  1600.0  1600.0  2060.0   
광주    C-유통      38.0  2411.842105   201.693576  1980.0  2280.0  2480.0   
경기수원  수원지동시장    76.0  1688.684211   177.556317  1350.0  1562.5  1715.0   
전북전주  전주남부시장    57.0  1486.666667   157.665257  1330.0  1330.0  1430.0   
울산    A-유통      38.0  2595.263158    73.327729  2390.0  2590.0  2590.0   
세종    C-유통      11.0  2236.363636    41.778637  2200.0  2200.0  2200.0   
강원춘천  춘천중앙시장    38.0  1815.000000    15.201351  1800.0  1800.0  1815.0   

                              전일조사가격                                    \
                  75%     max  count         mean          std     min   
조사지역명 조사가격시장명                                                            
경남창원  C-유통     5480.0  5480.0   15.0  4250.000000  1571.009684  2200.0   
제주    C-유통     4972.5  4980.0   50.0  3133.200000  1399.203038  1980.0   
대전    C-유통     4980.0  4980.0   58.0  3059.310345  1359.483728  1680.0   
서울    A-유통     2590.0  5290.0   48.0  2848.333333  1344.465936  1490.0   
부산    D-유통     2980.0  4980.0   62.0  2613.870968  1154.171841  1380.0   
경북안동  C-유통     2830.0  4980.0   87.0  2822.873563  1161.740783  1680.0   
서울    C-유통     2580.0  5280.0   54.0  2285.925926   763.810009  1680.0   
대전    E-유통     2650.0  3950.0   50.0  2237.400000   686.451419  1390.0   
대구    C-유통     2780.0  4950.0   60.0  2472.833333   655.754428  1900.0   
광주    양동시장     25

In [456]:
market_df = df.groupby(['표준시장명', '표준품목명', '조사가격품목명', '표준품종명', '조사가격품종명']).mean()
market_df.reset_index(inplace=True)
market_df[market_df.표준품목명 == target_item]

,표준시장명,표준품목명,조사가격품목명,표준품종명,조사가격품종명,당일조사가격,전일조사가격
42,강릉중앙시장,쇠고기,쇠고기,갈비(수입),미국산갈비,2300.000000,2300.000000
43,강릉중앙시장,쇠고기,쇠고기,우둔(수입),미국산불고기,1800.000000,1800.000000
44,강릉중앙시장,쇠고기,쇠고기,우둔(수입),호주산불고기,1400.000000,1400.000000
134,경동시장,쇠고기,쇠고기,갈비(수입),미국산갈비,2260.000000,2260.000000
135,경동시장,쇠고기,쇠고기,갈비(수입),호주산갈비,2160.000000,2160.000000
136,경동시장,쇠고기,쇠고기,등심(수입),호주산등심,1450.000000,1450.000000
137,경동시장,쇠고기,쇠고기,우둔(수입),미국산불고기,1760.000000,1760.000000
138,경동시장,쇠고기,쇠고기,우둔(수입),호주산불고기,1400.000000,1400.000000
315,동문시장,쇠고기,쇠고기,갈비(수입),미국산갈비,2060.000000,2060.000000
316,동문시장,쇠고기,쇠고기,등심(수입),호주산등심,1600.000000,1600.000000


In [457]:
distri_df = df.groupby(['조사가격시장명', '표준품목명', '조사가격품목명', '표준품종명', '조사가격품종명']).mean()
distri_df.reset_index(inplace=True)
distri_df[distri_df.표준품목명 == target_item]

,조사가격시장명,표준품목명,조사가격품목명,표준품종명,조사가격품종명,당일조사가격,전일조사가격
51,A-유통,쇠고기,쇠고기,갈비(수입),미국산갈비,1778.571429,1756.666667
52,A-유통,쇠고기,쇠고기,갈비(수입),호주산갈비,2560.000000,2560.000000
53,A-유통,쇠고기,쇠고기,갈비살(수입),미국산갈비살,2713.129252,2728.095238
54,A-유통,쇠고기,쇠고기,등심(수입),호주산등심,5290.000000,5290.000000
55,A-유통,쇠고기,쇠고기,우둔(수입),미국산불고기,1775.714286,1861.428571
56,A-유통,쇠고기,쇠고기,우둔(수입),호주산불고기,2204.814815,2204.814815
232,C-유통,쇠고기,쇠고기,갈비(수입),미국산갈비,1738.518519,1695.555556
233,C-유통,쇠고기,쇠고기,갈비(수입),호주산갈비,2302.042254,2284.366197
234,C-유통,쇠고기,쇠고기,갈비살(수입),미국산갈비살,2855.000000,2852.727273
235,C-유통,쇠고기,쇠고기,등심(수입),호주산등심,5047.391304,5047.391304


## Distributor type by region and item

#### the number of distributor type by region and item

In [472]:
df = origin_df
df = df[(df.조사구분명 == '소비자가격')] #  & (df.조사일자 == '2017-01-04')]
df = df[[
         '표준품목명', '조사가격품목명', '표준품종명', '조사가격품종명', # item category
         '조사지역명', '조사가격시장명' # region, market, distributor
        ]]
tmp_df = df.drop_duplicates()

In [473]:
result = tmp_df.groupby(['조사지역명', '표준품목명', '조사가격품목명', '표준품종명', '조사가격품종명']).count()
result

조사가격시장명
조사지역명 표준품목명    조사가격품목명 표준품종명     조사가격품종명           
강원    감        단감      부유        일반               1
      감귤       감귤      조생귤       일반               1
      감자       감자      수미        수미               1
      건가공(포)류  건오징어    오징어포      일반               1
      건가공(해조)류 건미역     자연미역      일반               1
      건제품      건포도     건포도(수입)   수입               1
      고구마      고구마(밤)  밤고구마      고구마(일반)          1
      고등어류     고등어     망치고등어     염장               1
      고추       붉은고추    홍고추       일반               1
               풋고추     꽈리고추      꽈리고추             1
                       청양        청양고추             1
                       청초        일반고추             1
      굴류       굴       굴(참굴)     일반               1
      깻잎       깻잎      깻잎(일반)    일반               1
      꽁치류      꽁치      꽁치(수입)    수입냉동             1
      닭고기      닭고기     통닭        도계               1
      당근       당근      흙당근       무세척              1
      대파       파       대파(일반)    대파               1
      돈육       돼지고기    갈비        갈비               1
                       목심살       목살               1
                       삼겹살       삼겹살(국산냉장)        1
                       앞다리살      앞다리살             1
      딸기       딸기      설향        일반               1
      땅콩       땅콩      땅콩(일반)    국산               1
      레몬       레몬      레몬(수입)    수입               1
      마늘       마늘      깐마늘       깐마늘(국산)          1
      메론       멜론      네트계       일반               1
      무        무       월동무       월동               1
      바나나      바나나     바나나(수입)   수입               1
      방울토마토    방울토마토   기타        국산               1
      배        배       신고        신고               1
      배추       배추      월동배추      월동               1
      벼        쌀       일반계       일반계              1
      블루베리     건블루베리   건블루베리(수입) 수입               1
      사과       사과      후지        후지               1
      상추       상추      적상추       적                1
      새송이      새송이버섯   새송이버섯     일반               1
      새우류      새우      흰다리새우     일반               1
      생강       생강      생강(일반)    국산               1
      쇠고기      쇠고기     갈비(수입)    호주산갈비            1
                       갈비살(수입)   미국산갈비살           1
                       우둔(수입)    호주산불고기           1
      아몬드      아몬드     아몬드(수입)   일반               1
      양배추      양배추     양배추(일반)   일반               1
      양파       양파      기타        양파(일반)           1
      열무       열무      열무(일반)    일반               1
      오렌지      오렌지     네블        네이블              1
      오이       오이      백다다기      다다기계통            1
                       취청        취청               1
      우유       우유      우유(일반)    일반               1
      전복류      전복      전복        일반               1
      젓갈(액젓)류  멸치액젓    멸치액젓      일반               1
      젓갈(일반)류  새우젓     새우젓       일반               1
      조기류      조기      참조기       수입부세             1
      조미제품     고춧가루    고추가루      일반               1
      쪽파       파       기타        쪽파               1
      참다래(키위)  참다래     기타        국산               1
                       키위(수입)    수입               1
      찹쌀       찹쌀      일반계       일반찹쌀             1
      천연조미료류   굵은소금    천일염       일반               1
      콩        콩       백태        백태(국산)           1
      파인애플     파인애플    파인애플(수입)  수입산              1
      파프리카     파프리카    기타        일반               1
      팥        팥(적두)   붉은팥       국산               1
      팽이버섯     팽이버섯    팽이1호      일반               1
      포도       포도      포도(수입)    수입포도             1
      피망(단고추)  피망      청피망       청피망              1
      한우       쇠고기     등심        한우등심             1
                       안심        안심               1
                       양지        양지               1
      호두       호두      호두(일반)    일반               1
      호박       호박      애호박       애호박              1
                       쥬키니호박     쥬키니              1
강원강릉  갈치류      갈치      은갈치       냉동               1
                                 생선               1
      감        단감      부유        일반     

In [7]:
import pandas as pd
coef_df = pd.read_csv("https://production-bobsim.s3.ap-northeast-2.amazonaws.com/food_material_price_predict_model/coef.csv", encoding='euc-kr')

In [10]:
coef_df.T

,0
unit,-0.005720
item_name_갈치류갈치은갈치냉동,0.981973
item_name_갈치류갈치은갈치생선,1.005253
item_name_감자감자수미수미,-0.828130
item_name_건가공(포)류건오징어오징어포일반,1.505273
item_name_건가공(해조)류건미역자연미역일반,-0.742582
item_name_건제품건포도건포도(수입)수입,-0.556467
item_name_고구마고구마(밤)밤고구마고구마(일반),-0.656256
item_name_고등어류고등어고등어(수입)수입,-0.020501
item_name_고등어류고등어망치고등어냉동,-0.074651
